# Import Libraries

In [ ]:
import re
import os
from bs4 import BeautifulSoup   
import pandas as pd
import urllib.request
from datetime import datetime, timedelta
import math
from time import strptime

# Input the variables

In [ ]:
##### Inputs######################

#Hur = input('Enter the Hurrican/Storm Name as it appears in NHC website (All Caps letter)')

Hur = 'COLIN'

#year = input("Enter the Year")

year = '2022'
stormNumber = 1
#########################

# Create Directory for the Storm

# Parent Directory path

parent_dir = os.getcwd()

directory_name = str(year) + "_" + Hur

path = os.path.join(parent_dir, directory_name)

if os.path.isdir(directory_name) is True:
    
    print("Directory already exist")
    
else:
    
    os.mkdir(path)
    

    print(f"Directory {directory_name} is created at {path}")


Adv_FH = input("Advisory Number: ")

Rmax_Scaling = input("Scale Rmax by: ")


url = "http://www.nhc.noaa.gov/archive/{0}/{1}.shtml?".format(str(year),Hur)

page = urllib.request.urlopen(url)

soup = BeautifulSoup(page,"lxml")

soup.unicode

# Calculations

In [ ]:
a = soup.find_all('a',href = re.compile("fstadv"))
key = 0
links = []
for row in a:
    print(a[key])
    link = "http://www.nhc.noaa.gov/" + a[key]['href']
    links.append(link)
    key+=1

#     if final_forecast is True:
#         a = a[:-1]
# except:
#     for row in a[:-1]:
#         print(a[key])
#         link = "http://www.nhc.noaa.gov/" + a[key]['href']
#         links.append(link)
#         key+=1
    
## IF NHC TAKES A LONG TIME TO UPLOAD TRACK DATA TO ARCHIVES, ADD THE FORECAST
## ADVISORY URL AND UNCOMMENT BELOW:
    
#link = "https://www.nhc.noaa.gov/text/refresh/MIATCMAT4+shtml/132038.shtml"
#links.append(link)

#Find all previous observed locations and windspeeds
#CP = Central Pressure

track = pd.DataFrame(columns = ['NOTES','Latitude','Longitude','Hour','Minutes','WindMax','CP','Day','Year','Month'],dtype=object)
########### DAB: Added "dtype=object" to allow flexible data types in data frame, and all "advisory" variables below
seq = 0
for link in links:
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page,"lxml")
    text = soup.find_all("pre")
    for row in text:
        adv = row.string
   
    init_wnd_idx = adv.find("MAX SUSTAINED WINDS")
    init_wind = int(round(float(adv[init_wnd_idx+19:init_wnd_idx+24])*1.151/5,0)*5)
    init_rd_idx = adv.find("CENTER LOCATED NEAR")
    init_lat = float(adv[init_rd_idx+20:init_rd_idx+24])
    init_lon = float(adv[init_rd_idx+26:init_rd_idx+31])*-1
    init_time_idx = adv.find("UTC")
    init_time = int(adv[init_time_idx-5:init_time_idx-3])
    init_min = int(adv[init_time_idx-3:init_time_idx-1])
    init_month = strptime(adv[init_time_idx+8:init_time_idx+11],'%b').tm_mon
    init_day = int(adv[init_time_idx+12:init_time_idx+14])
    init_year = int(adv[init_time_idx+15:init_time_idx+20])
    init_pressure_idx = adv.find("ESTIMATED MINIMUM CENTRAL PRESSURE")
    init_pressure = int(adv[init_pressure_idx+35:init_pressure_idx+39])
    init_adv_idx = adv.find("ADVISORY NUMBER")
    init_adv = int(adv[init_adv_idx+17:init_adv_idx+19])
    init_total_Time = (init_month-1)*31*24 + init_day*24 + init_time
    advisory = pd.DataFrame({'NOTES': [init_adv],'Latitude': [init_lat],'Longitude': [init_lon],'Hour': [init_time],'Minutes': [init_min],'WindMax': [init_wind],'CP': [init_pressure], 'Day': [init_day],'Year': [init_year], 'Month': [init_month]},dtype=object)
    frames = [track,advisory]
    track = pd.concat(frames,ignore_index=True)

#Get Central Pressure Adjustment Factor
AdvCP =  track["CP"][key-1]
AdvVmax = track["WindMax"][key-1]
EstimatedCP = (1013.25 + (3.6588 - (AdvVmax/1.151)*0.483 - math.pow(((AdvVmax/1.151) / 24.254),2) )) 
CPcorrectionFactor = AdvCP / EstimatedCP
final_forecast = adv.find("THIS IS THE LAST FORECAST/ADVISORY ISSUED BY THE NATIONAL HURRICANE")

#Get latest advisory with forecast track    
furturelink = links[len(links)-1]
page = urllib.request.urlopen(link)
soup = BeautifulSoup(page,"lxml")
text = soup.find_all("pre")

##Commented text is no longer necessary 
#for row in text:
#    row_text = row.string.replace('...INLAND','')
#    row_text = row_text.replace('...OVER WATER','')
#    row_text = row_text.replace('...POST-TROP/INLAND','')
#    row_text = row_text.replace('...OVER ISLE OF YOUTH','')  ### DAB: ADDED THIS LINE FOR NEAR LANDFALL
#    row_text = row_text.replace('...INLAND OVER LONG ISLAND','')  ### DAB: ADDED THIS LINE FOR NEAR LANDFALL
#    row_text = row_text.replace(' OVER LONG ISLAND','')  ### DAB: ADDED THIS LINE FOR NEAR LANDFALL
#    row_text = row_text.replace('...POST-TROPICAL','')  ### DAB: ADDED THIS LINE FOR NEAR LANDFALL
#    row_text = row_text.replace('...NEAR COAST','')  ### DAB: ADDED THIS LINE FOR NEAR LANDFALL
#    row_text = row_text.replace('...POST-TROP/EXTRATROP','')  ### DAB: ADDED THIS LINE FOR POST LANDFALL
#    row_text = row_text.replace('...DISSIPATED','')  ### DAB: ADDED THIS LINE FOR POST LANDFALL
#    adv = row_text



#Find time of final observed location   
fin_time = str(init_month) + " " + str(init_day) + " " + str(init_year) + " " + str(int(init_time)) + " " + str(int(init_min))
################ DAB: Added int() around init_time above to remove decimal for .0
fin_time_dt = datetime.strptime(fin_time, '%m %d %Y %H %M')



#12 hour advisory


    
try:
    one_idx = adv.find("FORECAST VALID")
    one_lat = float(adv[one_idx+24:one_idx+28])
    one_lon = float(adv[one_idx+30:one_idx+35])*-1
    tmp_idx = adv.find("MAX WIND",one_idx)
    one_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5


    if (fin_time_dt.hour == 21) or (fin_time_dt.hour == 9):
        one_dt = fin_time_dt + timedelta(hours=9)
    else:
        one_dt = fin_time_dt + timedelta(hours=9)
    one_adv = "+12"
    one_time = one_dt.hour
    one_min = one_dt.minute
    one_pressure = (1013.25 + (3.6588 - (one_wind/1.151)*0.483 - math.pow(((one_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
    one_day = one_dt.day
    day1 = one_day
    one_year = one_dt.year
    one_month = one_dt.month
    advisory = pd.DataFrame({'NOTES': [one_adv],'Latitude': [one_lat],'Longitude': [one_lon],'Hour': [one_time],'Minutes': [one_min],'WindMax': [int(one_wind)],'CP': [one_pressure], 'Day': [one_day],'Year': [one_year], 'Month': [one_month]},dtype=object)
    ########### DAB: Added "dtype=object" to allow flexible data types in data frame above
    frames = [track,advisory]
    track = pd.concat(frames,ignore_index=True)
    two_idx = one_idx
    two_dt = one_dt;
    #Remaining advisories
    ct = 1
    hrCt = 12
    #for ii in range(0,7):
    while adv.find("FORECAST VALID",two_idx+1) != -1 & (adv.find("OUTLOOK VALID",two_idx+1) != -1 | adv.find("DISSIPATED",two_idx+1) != -1) :
        ct = ct + 1
        if (ct<7):
            two_idx = adv.find("FORECAST VALID",two_idx+1)
            if (adv[two_idx+26:two_idx+36] == "DISSIPATED"):
                break
            two_lat = float(adv[two_idx+24:two_idx+28])
            two_lon = float(adv[two_idx+30:two_idx+35])*-1
            tmp_idx = adv.find("MAX WIND",two_idx)
            two_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5
            hrCt = hrCt + 12
            two_dt = two_dt + timedelta(hours=12)
            two_time = two_dt.hour
            two_adv = "+" + str(hrCt)
            two_min = two_dt.minute
            two_pressure = (1013.25 + (3.6588 - (two_wind/1.151)*0.483 - math.pow(((two_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
            two_day = two_dt.day
            two_year = two_dt.year
            two_month = two_dt.month
            advisory = pd.DataFrame({'NOTES': [two_adv],'Latitude': [two_lat],'Longitude': [two_lon],'Hour': [two_time],'Minutes': [two_min],'WindMax': [int(two_wind)],'CP': [two_pressure], 'Day': [two_day],'Year': [two_year], 'Month': [two_month]},dtype=object)
            frames = [track,advisory]
            track = pd.concat(frames,ignore_index=True)
        else:
            two_idx = adv.find("OUTLOOK VALID",two_idx+1)
            if (adv[two_idx+25:two_idx+35] == 'DISSIPATED'):
                break
            two_lat = float(adv[two_idx+23:two_idx+27])
            two_lon = float(adv[two_idx+29:two_idx+34])*-1
            tmp_idx = adv.find("MAX WIND",two_idx)
            two_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5
            hrCt = hrCt + 24
            two_dt = two_dt + timedelta(hours=24)
            two_time = two_dt.hour
            two_adv = "+" + str(hrCt)
            two_min = two_dt.minute
            two_pressure = (1013.25 + (3.6588 - (two_wind/1.151)*0.483 - math.pow(((two_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
            two_day = two_dt.day
            two_year = two_dt.year
            two_month = two_dt.month
            advisory = pd.DataFrame({'NOTES': [two_adv],'Latitude': [two_lat],'Longitude': [two_lon],'Hour': [two_time],'Minutes': [two_min],'WindMax': [int(two_wind)],'CP': [two_pressure], 'Day': [two_day],'Year': [two_year], 'Month': [two_month]},dtype=object)
            frames = [track,advisory]
            track = pd.concat(frames,ignore_index=True)

except ValueError as ve:
    print('End of forecast')



In [ ]:
for link in links:
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page,"lxml")
    text = soup.find_all("pre")
    for row in text:
        adv = row.string
        #print(adv)
    final_forecast = ("THIS IS THE LAST FORECAST")
    while final_forecast in adv:
      
        break
#     if final_forecast is True:
#         a = a[:-1]
#     else:
#         print('Continue')

In [ ]:
try:
    one_idx = adv.find("FORECAST VALID")
    one_lat = float(adv[one_idx+24:one_idx+28])
    one_lon = float(adv[one_idx+30:one_idx+35])*-1
    tmp_idx = adv.find("MAX WIND",one_idx)
    one_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5


    if (fin_time_dt.hour == 21) or (fin_time_dt.hour == 9):
        one_dt = fin_time_dt + timedelta(hours=9)
    else:
        one_dt = fin_time_dt + timedelta(hours=9)
    one_adv = "+12"
    one_time = one_dt.hour
    one_min = one_dt.minute
    one_pressure = (1013.25 + (3.6588 - (one_wind/1.151)*0.483 - math.pow(((one_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
    one_day = one_dt.day
    day1 = one_day
    one_year = one_dt.year
    one_month = one_dt.month
    advisory = pd.DataFrame({'NOTES': [one_adv],'Latitude': [one_lat],'Longitude': [one_lon],'Hour': [one_time],'Minutes': [one_min],'WindMax': [int(one_wind)],'CP': [one_pressure], 'Day': [one_day],'Year': [one_year], 'Month': [one_month]},dtype=object)
    ########### DAB: Added "dtype=object" to allow flexible data types in data frame above
    frames = [track,advisory]
    track = pd.concat(frames,ignore_index=True)
    two_idx = one_idx
    two_dt = one_dt;
    #Remaining advisories
    ct = 1
    hrCt = 12
    #for ii in range(0,7):
    while adv.find("FORECAST VALID",two_idx+1) != -1 & (adv.find("OUTLOOK VALID",two_idx+1) != -1 | adv.find("DISSIPATED",two_idx+1) != -1) :
        ct = ct + 1
        if (ct<7):
            two_idx = adv.find("FORECAST VALID",two_idx+1)
            if (adv[two_idx+26:two_idx+36] == "DISSIPATED"):
                break
            two_lat = float(adv[two_idx+24:two_idx+28])
            two_lon = float(adv[two_idx+30:two_idx+35])*-1
            tmp_idx = adv.find("MAX WIND",two_idx)
            two_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5
            hrCt = hrCt + 12
            two_dt = two_dt + timedelta(hours=12)
            two_time = two_dt.hour
            two_adv = "+" + str(hrCt)
            two_min = two_dt.minute
            two_pressure = (1013.25 + (3.6588 - (two_wind/1.151)*0.483 - math.pow(((two_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
            two_day = two_dt.day
            two_year = two_dt.year
            two_month = two_dt.month
            advisory = pd.DataFrame({'NOTES': [two_adv],'Latitude': [two_lat],'Longitude': [two_lon],'Hour': [two_time],'Minutes': [two_min],'WindMax': [int(two_wind)],'CP': [two_pressure], 'Day': [two_day],'Year': [two_year], 'Month': [two_month]},dtype=object)
            frames = [track,advisory]
            track = pd.concat(frames,ignore_index=True)
        else:
            two_idx = adv.find("OUTLOOK VALID",two_idx+1)
            if (adv[two_idx+25:two_idx+35] == 'DISSIPATED'):
                break
            two_lat = float(adv[two_idx+23:two_idx+27])
            two_lon = float(adv[two_idx+29:two_idx+34])*-1
            tmp_idx = adv.find("MAX WIND",two_idx)
            two_wind = round(float(adv[tmp_idx+8:tmp_idx+13])*1.151/5,0)*5
            hrCt = hrCt + 24
            two_dt = two_dt + timedelta(hours=24)
            two_time = two_dt.hour
            two_adv = "+" + str(hrCt)
            two_min = two_dt.minute
            two_pressure = (1013.25 + (3.6588 - (two_wind/1.151)*0.483 - math.pow(((two_wind/1.151) / 24.254),2) )) *CPcorrectionFactor
            two_day = two_dt.day
            two_year = two_dt.year
            two_month = two_dt.month
            advisory = pd.DataFrame({'NOTES': [two_adv],'Latitude': [two_lat],'Longitude': [two_lon],'Hour': [two_time],'Minutes': [two_min],'WindMax': [int(two_wind)],'CP': [two_pressure], 'Day': [two_day],'Year': [two_year], 'Month': [two_month]},dtype=object)
            frames = [track,advisory]
            track = pd.concat(frames,ignore_index=True)

except ValueError as ve:
    print('End of forecast')


# Adjusting the Varaibles

In [ ]:
#Get Central Pressure Adjustment Factor
AdvIdx = key-1
AdvCP =  track["CP"][AdvIdx]
AdvVmax = track["WindMax"][AdvIdx]

adv_lat0 = track['Latitude'][AdvIdx-1]
adv_lon0 = track['Longitude'][AdvIdx-1]

adv_lat = track['Latitude'][AdvIdx]
adv_lon = track['Longitude'][AdvIdx]

timeElapsed = (track.iloc[AdvIdx]['Month'] - track.iloc[AdvIdx-1]['Month'])*24*31 + (track.iloc[AdvIdx]['Day'] - track.iloc[AdvIdx-1]['Day'])*24 + (track.iloc[AdvIdx]['Hour'] - track.iloc[AdvIdx-1]['Hour'])

p = 0.017453292519943295
a = 0.5 - math.cos((adv_lat-adv_lat0)*p)/2 + math.cos(adv_lat0*p)*math.cos(adv_lat*p) * (1-math.cos((adv_lon0-adv_lon)*p)) / 2
dist = 7917.509 * math.asin(math.sqrt(a))   ## miles
fwdspd = dist/timeElapsed 

Vsrm = ((AdvVmax/1.15)-1.5*(fwdspd/1.15)**(0.62)) ## kts
EstimatedCP = 23.286-0.483*(Vsrm)-((AdvVmax/24.254)**2.)-12.587*0.49-0.483*adv_lat+1013.25
CPcorrectionFactor = AdvCP / EstimatedCP

start = key
end = len(track)

for i in range(start,end): 
    lat0 = track['Latitude'][i-1]
    lon0 = track['Longitude'][i-1]

    lat1 = track['Latitude'][i]
    lon1 = track['Longitude'][i]

    windmax = track['WindMax'][i]
    timeElapsed = (track['Month'][i] - track['Month'][i-1])*24*31 + (track['Day'][i] - track['Day'][i-1])*24 + (track['Hour'][i] - track['Hour'][i-1])

    p = 0.017453292519943295
    a = 0.5 - math.cos((lat1-lat0)*p)/2 + math.cos(lat0*p)*math.cos(lat1*p) * (1-math.cos((lon1-lon0)*p)) / 2
    dist = 7917.509 * math.asin(math.sqrt(a))   ## miles
    fwdspd = dist/timeElapsed

    Vsrm = ((windmax/1.15)-1.5*(fwdspd/1.15)**(0.62)) ## kts
    CP = int(23.286-0.483*(Vsrm)-((windmax/24.254)**2.)-12.587*0.49-0.483*lat1+1013.25)
    
    track.at[i, 'CP'] = int(CP*CPcorrectionFactor)

seq = 0
#Calculate Rmax for each track postition
track['Rmax'] = 1
for wind,lat in zip(track['WindMax'],track['Latitude']):
    #track['Rmax'][seq] = max((218.3784-1.2014*float(wind)+ \
    #             math.pow((float(wind)/10 documen.9844),2)-math.pow((float(wind)/35.3052),3)- \
    #             145.509*math.cos(float(lat)*22/1260))/1.15,10)
    
    if (float(wind) < 157):
        track['Rmax'][seq] = int(max((41.883 * math.exp(0.0169 * float(lat)-0.0089*float(wind)))*float (Rmax_Scaling),6))
        #track[('Rmax', seq)] = int(max((41.883 * math.exp(0.0169 * float(lat)-0.0089*float(wind)))*float (Rmax_Scaling),6))
    else:
        track['Rmax'][seq] = int(max((2521.43 * math.exp(0.0169 * float(lat)-0.035*float(wind)))*float (Rmax_Scaling),6))
        #track[:,('Rmax', seq)] = int(max((2521.43 * math.exp(0.0169 * float(lat)-0.035*float(wind)))*float (Rmax_Scaling),6))

#    track['Rmax'][seq] = max((43.2 - 0.234 * float(wind)) * math.exp(0.0169 * float(lat)),5)
    seq+=1
    
track['Name'] = Hur
track['Storm#'] = stormNumber
#track['Min'] = 0
track['SeasonYear'] = year
#track['Direction'] = 0

##Calculate forward speed of the track at each point
#seq = 0
#track['Fspeed'] = 1.0
#for lat,lon in zip(track['Latitude'],track['Longitude']):
#    if seq == len(track)-1:
#        break
#    track['Fspeed'][seq] = (3959*math.acos(math.cos(math.radians(90-float(lat)))* \
#                     math.cos(math.radians(90-float(track['Latitude'][seq+1]))) + \
#                     math.sin(math.radians(90-float(lat))) * \
#                     math.sin(math.radians(90-float(track['Latitude'][seq+1]))) * \
#                     math.cos(math.radians(float(track['Longitude'][seq+1])-float(track['Longitude'][seq]))))) / (int(track['SEQ'][seq+1])-int(track['SEQ'][seq]))
#                     
#    time = (int(track['SEQ'][seq+1])-int(track['SEQ'][seq])) 
#    seq+=1

In [ ]:
AdvIdx = key-1
AdvCP =  track["CP"][AdvIdx]
AdvVmax = track["WindMax"][AdvIdx]

adv_lat0 = track['Latitude'][AdvIdx-1]
adv_lon0 = track['Longitude'][AdvIdx-1]

adv_lat = track['Latitude'][AdvIdx]
adv_lon = track['Longitude'][AdvIdx]

print(AdvCP)

In [ ]:
track

# Storing the results

In [ ]:
#Finalize and save track
track = track[['SeasonYear', 'Storm#', 'Name', 'Year', 'Month', 'Day', 'Hour', 'Minutes', 'Latitude', 'Longitude', 'WindMax', 'CP', 'Rmax', 'NOTES']]

TrackFileName = "\\" + directory_name + "\KCC_TrackFile_"+ str(year) + "_{0}_Adv_{1}.txt".format(Hur, str (Adv_FH))
track.to_csv(r"C:\Users\smahmud\Documents"+ TrackFileName,"\t",index=False)

print( f"Hurricane {Hur} save successfully at {path} ")


